# Logistic and SVM Model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
import xgboost as xgb
import sklearn.datasets
import gc
import warnings
warnings.filterwarnings('ignore')

In [19]:
train_X = pd.read_csv('new_data/new_train_X.csv')
val_X = pd.read_csv('new_data/new_valid_X.csv')
train_y = pd.read_csv('new_data/new_train_y.csv')
val_y = pd.read_csv('new_data/new_valid_y.csv')

In [20]:
train_X.head()

,Unnamed: 0,Unnamed: 0.1,msno,song_id,source_screen_name,source_system_tab,source_type,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,time,registration_init_time_int,expiration_date_int
0,4014890,7039846,785,141648,18,7,7,219454.0,371,44988,20734,4032,2,0,0,2,2,0.708647,4155,6465
1,865446,2067552,9176,156565,16,6,8,270001.0,127,8947,74276,26024,9,0,0,2,2,0.208124,4384,6487
2,1151370,5503771,31040,43786,11,0,7,265822.0,371,40039,84290,35947,2,12,33,1,3,0.554022,2890,6479
3,575108,639057,26412,387047,8,3,4,266031.0,364,42827,71260,26024,2,0,0,2,2,0.064329,6153,6486
4,460130,6368926,24075,136842,4,0,7,280346.0,364,43072,81070,31877,2,0,0,2,3,0.641111,5634,6874


In [21]:
train_y.head()

,target
0,0.0
1,1.0
2,0.0
3,1.0
4,1.0


In [22]:
print(train_X.shape,train_y.shape)
print(val_X.shape,val_y.shape)

(552909, 20) (552909, 1)
(92152, 20) (92152, 1)


In [23]:
catCols = ['msno', 'song_id', 'source_screen_name', 'source_system_tab', 'source_type', 'genre_ids', 'artist_name', 
           'composer', 'lyricist', 'language', 'city', 'gender', 'registered_via']
numCols = ['bd', 'song_length','Unnamed: 0','Unnamed: 0.1','time','registration_init_time_int','expiration_date_int']

## Logistic Regression

In [17]:
#debugging with tiny data set
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

clf.fit(train_tiny_X, train_tiny_y['target'])
print("Model score on training set:")
print("model score: %.3f" % clf.score(train_tiny_X, train_tiny_y['target']))
print(classification_report(train_tiny_y['target'], clf.predict(train_tiny_X)))
print(roc_auc_score(train_tiny_y['target'], clf.predict_proba(train_tiny_X)[:,1]))

/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.633
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      4985
         1.0       0.93      0.94      0.93      5015

   micro avg       0.93      0.93      0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000

0.9814643531791786


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [18]:
print("Model score on validation set:")
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(val_tiny_y['target'], clf.predict(val_tiny_X)))
print(roc_auc_score(val_tiny_y['target'], clf.predict_proba(val_tiny_X)[:,1]))

model score: 0.633
              precision    recall  f1-score   support

         0.0       0.62      0.62      0.62       477
         1.0       0.65      0.65      0.65       523

   micro avg       0.63      0.63      0.63      1000
   macro avg       0.63      0.63      0.63      1000
weighted avg       0.63      0.63      0.63      1000

0.6737737051601188


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [46]:
# DON'T RUN IT YET!!!!!! HUGE DATASET!!!

# numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
# categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
# preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
#                                                ('cat', categorical_transformer, catCols)])
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', LogisticRegression())])

# clf.fit(train_X, train_y['target'])
# print("model score: %.3f" % clf.score(val_X, val_y['target']))
# print(classification_report(val_y['target'], clf.predict(val_X)))
# print(roc_auc_score(val_y, clf.predict_proba(val_X)['target']))

Default: Logistic Regression with No Penalty, learning rate = 'optimal'  
val acc = 0.66

In [28]:
#mini batch training with full data and early stopping
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf_log = Pipeline(steps=[('preprocessor', preprocessor),
                ('classifier', SGDClassifier(loss='log', penalty=None, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                max_iter=1000, tol=0.001, shuffle=True, verbose=1, epsilon=0.1, n_jobs=None, 
                                random_state=None, learning_rate='adaptive', eta0=0.001, 
                                power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, 
                                class_weight=None, warm_start=True, average=False))])
#default, no penalty
clf_log.fit(train_X, train_y['target'])
print("model score: %.3f" % clf_log.score(val_X, val_y['target']))
print(classification_report(val_y['target'], clf_log.predict(val_X)))
print(roc_auc_score(val_y['target'], clf_log.predict_proba(val_X)[:,1]))
#val acc 0.660


-- Epoch 1
Norm: 2.36, NNZs: 172997, Bias: -0.003480, T: 497618, Avg. loss: 0.574006
Total training time: 0.34 seconds.
-- Epoch 2
Norm: 3.58, NNZs: 172997, Bias: -0.004140, T: 995236, Avg. loss: 0.568198
Total training time: 0.78 seconds.
-- Epoch 3
Norm: 4.65, NNZs: 172997, Bias: -0.004343, T: 1492854, Avg. loss: 0.565144
Total training time: 1.21 seconds.
-- Epoch 4
Norm: 5.62, NNZs: 172997, Bias: -0.004314, T: 1990472, Avg. loss: 0.562776
Total training time: 1.63 seconds.
-- Epoch 5
Norm: 6.55, NNZs: 172997, Bias: -0.004610, T: 2488090, Avg. loss: 0.560732
Total training time: 2.07 seconds.
-- Epoch 6
Norm: 7.42, NNZs: 172997, Bias: -0.004966, T: 2985708, Avg. loss: 0.558962
Total training time: 2.50 seconds.
-- Epoch 7
Norm: 7.59, NNZs: 172997, Bias: -0.004683, T: 3483326, Avg. loss: 0.557016
Total training time: 2.94 seconds.
-- Epoch 8
Norm: 7.76, NNZs: 172997, Bias: -0.004715, T: 3980944, Avg. loss: 0.556677
Total training time: 3.37 seconds.
-- Epoch 9
Norm: 7.92, NNZs: 17299

In [24]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf_svm = Pipeline(steps=[('preprocessor', preprocessor),
                ('classifier', SGDClassifier(loss='hinge', penalty=None, early_stopping=True, 
                                             validation_fraction=0.1, n_iter_no_change=5))])
#default, no penalty
clf_svm.fit(train_X, train_y['target'])
print("model score: %.3f" % clf_svm.score(val_X, val_y['target']))
print(classification_report(val_y['target'], clf_svm.predict(val_X)))
print(roc_auc_score(val_y['target'], clf_svm.predict(val_X)>0.5))

model score: 0.655
              precision    recall  f1-score   support

         0.0       0.64      0.68      0.66     45690
         1.0       0.67      0.63      0.65     46462

    accuracy                           0.66     92152
   macro avg       0.66      0.66      0.66     92152
weighted avg       0.66      0.66      0.66     92152

0.6555275025289365


best SVM model(with no penalty):  
learning rate = 'adaptive', eta = 0.01  
best val acc = 0.697

In [25]:
etas = [0.01]
best_model_svm = None
best_model_score_svm = 0.0
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
for eta in etas:
    clf_svm = Pipeline(steps=[('preprocessor', preprocessor),
                    ('classifier', SGDClassifier(loss='hinge', penalty=None, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                    max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                                    random_state=None, learning_rate='adaptive', eta0=eta, 
                                    power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, 
                                    class_weight=None, warm_start=True, average=False))])
    clf_svm.fit(train_X, train_y['target'])
    model_score = clf_svm.score(val_X, val_y['target'])
    if model_score > best_model_score_svm:
        best_model_score_svm = model_score
        best_model_svm = clf_svm
        print('eta0:',eta)
        print('best val acc:',best_model_score_svm)
                                        
# print("best model score:" ,best_model_score_svm)
# print(classification_report(val_y['target'], best_model_svm.predict(val_X)))
# print(roc_auc_score(val_y['target'], best_model_svm.predict(val_X)>0.5))
#print(roc_auc_score(val_y['target'], clf_svm.predict_proba(val_X)[:,1]))

eta0: 0.01
best val acc: 0.6872449865439708


In [27]:
print("best model score:" ,best_model_score_svm)
print(classification_report(val_y['target'], best_model_svm.predict(val_X)))
print(roc_auc_score(val_y['target'], best_model_svm.predict(val_X)>0.5))

best model score: 0.6872449865439708
              precision    recall  f1-score   support

         0.0       0.69      0.68      0.68     45690
         1.0       0.69      0.69      0.69     46462

    accuracy                           0.69     92152
   macro avg       0.69      0.69      0.69     92152
weighted avg       0.69      0.69      0.69     92152

0.6872018519378474


best logistic regression model(with no penalty):  
learning rate = 'adaptive', eta = 0.01  
best val acc = 0.694

In [69]:
#fine tune learning rate on logistic regression
best_model_df = None
best_model_score_df = 0.0
learning_rate = ['optimal','invscaling','adaptive']
etas = [0.01,0.001,0.0001]
for lr in learning_rate:
    for eta in etas:
        clf_log_df = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', SGDClassifier(loss='log', penalty=None, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                        max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                                        random_state=None, learning_rate=lr, eta0=eta, 
                                        power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, 
                                        class_weight=None, warm_start=True, average=False))])
        clf_log_df.fit(train_X, train_y['target'])
        model_score = clf_log_df.score(val_X, val_y['target'])
        if model_score > best_model_score_df:
            best_model_score_df = model_score
            best_model_df = clf_log_df
            print('learning_rate:',lr,'eta0:',eta)
            print('best val acc:',best_model_score_df)
                                        
print("best model score:" ,best_model_score_df)
print(classification_report(val_y['target'], best_model_df.predict(val_X)))
print(roc_auc_score(val_y['target'], best_model_df.predict_proba(val_X)[:,1]))

learning_rate: optimal eta0: 0.01
best val acc: 0.6600669214570903
learning_rate: optimal eta0: 0.0001
best val acc: 0.6603908880004283
learning_rate: adaptive eta0: 0.01
best val acc: 0.694752351976772
best model score: 0.694752351976772
              precision    recall  f1-score   support

         0.0       0.69      0.69      0.69    732500
         1.0       0.70      0.70      0.70    742961

   micro avg       0.69      0.69      0.69   1475461
   macro avg       0.69      0.69      0.69   1475461
weighted avg       0.69      0.69      0.69   1475461

0.763170012584044


In [26]:
#fine tune SVM and Logistic regression with penalty
best_model = None
best_model_score = 0.0
loss = ['log', 'hinge']
penalty = ['l1','l2','elasticnet']

for ls in loss:
    for plty in penalty: 
        clf_log_tune = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', SGDClassifier(loss=ls, penalty=plty, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                        max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                                        random_state=None, learning_rate='adaptive', eta0=0.01, 
                                        power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, 
                                        class_weight=None, warm_start=True, average=False))])
        clf_log_tune.fit(train_X, train_y['target'])
        model_score = clf_log_tune.score(val_X, val_y['target'])
        print( 'loss:', ls, 'penalty:', plty,'val acc:', model_score)
        if model_score > best_model_score:
            best_model_score = model_score
            best_model = clf_log_tune
            print( 'best loss:', ls, 'best penalty:', plty,'best val acc:', model_score)

print("best model score:" ,best_model_score)
print(classification_report(val_y['target'], best_model.predict(val_X)))
print(roc_auc_score(val_y['target'], best_model.predict_proba(val_X)[:,1]))

loss: log penalty: l1 val acc: 0.6449670110252627
best loss: log best penalty: l1 best val acc: 0.6449670110252627
loss: log penalty: l2 val acc: 0.6635016060421912
best loss: log best penalty: l2 best val acc: 0.6635016060421912
loss: log penalty: elasticnet val acc: 0.6547443354457853
loss: hinge penalty: l1 val acc: 0.6381087768035419
loss: hinge penalty: l2 val acc: 0.6457049223022832
loss: hinge penalty: elasticnet val acc: 0.641071273548051
best model score: 0.6635016060421912
              precision    recall  f1-score   support

         0.0       0.67      0.64      0.65     45690
         1.0       0.66      0.68      0.67     46462

    accuracy                           0.66     92152
   macro avg       0.66      0.66      0.66     92152
weighted avg       0.66      0.66      0.66     92152

0.7247404866963721


# SVM with kernel

In [1]:
#rbf
from sklearn.kernel_approximation import RBFSampler
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
rbf_feature = RBFSampler(gamma=0.2, random_state=1)
clf_kernel = Pipeline(steps=[('preprocessor', preprocessor),
                ('kernel',rbf_feature),
                ('classifier', SGDClassifier(loss='hinge', penalty=None, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                max_iter=1000, tol=0.001, shuffle=True, verbose=1, epsilon=0.1, n_jobs=None, 
                                random_state=None, learning_rate='adaptive', eta0=0.01, 
                                power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5))])
#default, no penalty
clf_kernel.fit(train_tiny_X, train_tiny_y['target'])
print("model score: %.3f" % clf_kernel.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(val_tiny_y['target'], clf_kernel.predict(val_tiny_X)))
#print(roc_auc_score(val_tiny_y['target'], clf_kernel.predict_proba(val_tiny_X)[:,1]))



NameError: name 'Pipeline' is not defined

## Logistic Regression with Regularization

### L1

In [19]:
from sklearn.feature_selection import SelectKBest
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(penalty='l1'))])

clf.fit(train_tiny_X, train_tiny_y['target'])
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(train_tiny_y['target'], clf.predict(train_tiny_X)))
print(roc_auc_score(train_tiny_y['target'], clf.predict_proba(train_tiny_X)[:,1]))

/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.660
              precision    recall  f1-score   support

         0.0       0.72      0.69      0.71      4985
         1.0       0.71      0.73      0.72      5015

   micro avg       0.71      0.71      0.71     10000
   macro avg       0.71      0.71      0.71     10000
weighted avg       0.71      0.71      0.71     10000

0.7882900546104915


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [20]:
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(val_tiny_y['target'], clf.predict(val_tiny_X)))
print(roc_auc_score(val_tiny_y['target'], clf.predict_proba(val_tiny_X)[:,1]))

model score: 0.660
              precision    recall  f1-score   support

         0.0       0.64      0.65      0.65       477
         1.0       0.68      0.67      0.67       523

   micro avg       0.66      0.66      0.66      1000
   macro avg       0.66      0.66      0.66      1000
weighted avg       0.66      0.66      0.66      1000

0.6840073595728562


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


## L2

In [21]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(penalty='l2'))])

clf.fit(train_tiny_X, train_tiny_y['target'])
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(train_tiny_y['target'], clf.predict(train_tiny_X)))
print(roc_auc_score(train_tiny_y['target'], clf.predict_proba(train_tiny_X)[:,1]))

/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.633
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      4985
         1.0       0.93      0.94      0.93      5015

   micro avg       0.93      0.93      0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000

0.9814643531791786


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [22]:
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(val_tiny_y['target'], clf.predict(val_tiny_X)))
print(roc_auc_score(val_tiny_y['target'], clf.predict_proba(val_tiny_X)[:,1]))

model score: 0.633
              precision    recall  f1-score   support

         0.0       0.62      0.62      0.62       477
         1.0       0.65      0.65      0.65       523

   micro avg       0.63      0.63      0.63      1000
   macro avg       0.63      0.63      0.63      1000
weighted avg       0.63      0.63      0.63      1000

0.6737737051601188


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


## Gradient Boosting & Decision Tree
Moved to "Tree & Boosting" Notebook

## Lasso

In [26]:
from sklearn.linear_model import Lasso
alphas = [0.01,0.05,0.1,0.2]
for a in alphas:
    print('alpha: {}'.format(a))
    lasso = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor',  Lasso(alpha = a))])
    lasso.fit(train_tiny_X, train_tiny_y['target'])
    print(np.sqrt(metrics.mean_squared_error(val_tiny_y['target'], lasso.predict(val_tiny_X))))

alpha: 0.01


/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype i

0.48453905786700674
alpha: 0.05


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


0.49497671219414635
alpha: 0.1


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


0.49993324554384255
alpha: 0.2
0.49993324554384255


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [ ]:
gc.collect()